<a href="https://colab.research.google.com/github/Hanklin999/SQL_Greenvines_CRM_Customer_Retention_byAppWorks/blob/main/SQL_Greenvines_CRM_Customer_Retention_byAppWorks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greenvines CRM Customer Retention – SQL Case Study

This notebook analyzes new customer return behavior for **Greenvines**, a clean beauty brand, using structured SQL queries and real business metrics.  
We aim to explore **year-over-year retention rate changes by acquisition channel**, in support of Greenvines' customer growth strategy.

---

#### Table of Contents
1. Dataset Upload
2. Load CSVs into DataFrames
3. Create SQLite In-Memory Database
4. Upload Tables to SQL
5. Retention Rate Query (Channel × Year)
6. Business Insights & Recommendations



##  Step 1: Load CSVs into Pandas DataFrames

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving table 5. Channels.csv to table 5. Channels.csv
Saving table 4. Products.csv to table 4. Products.csv
Saving table 3. OrderDetails.csv to table 3. OrderDetails.csv
Saving table 1. Customers.csv to table 1. Customers.csv
Saving table 2. Orders.csv to table 2. Orders.csv


In [ ]:
import os

os.rename('table 1. Customers.csv', 'Customers.csv')
os.rename('table 2. Orders.csv', 'Orders.csv')
os.rename('table 3. OrderDetails.csv', 'OrderDetails.csv')
os.rename('table 4. Products.csv', 'Products.csv')
os.rename('table 5. Channels.csv', 'Channels.csv')

# Check current directory
print(os.getcwd())

# List all files in current directory
print(os.listdir())



/content
['.config', 'Orders.csv', 'Customers.csv', 'OrderDetails.csv', 'Products.csv', 'Channels.csv', 'drive', 'sample_data']


##  Step 2: Load CSVs into Pandas DataFrames

We load all five tables into DataFrames before inserting them into the SQLite database.


In [ ]:
import pandas as pd

# Load each CSV into a DataFrame
customers = pd.read_csv('Customers.csv')
orders = pd.read_csv('Orders.csv')
order_details = pd.read_csv('OrderDetails.csv')
products = pd.read_csv('Products.csv')
channels = pd.read_csv('Channels.csv')

# Optional preview
customers.head()


,CustomerId,FirstTransactionDate,FirstChannel,FirstTransactionYear
0,c551b230c3867,2021/01/01,台中勤美門市,2021
1,ca71a4d0c8b1e,2021/01/01,台中勤美門市,2021
2,c6cfd580c8537,2021/01/01,台北京站門市,2021
3,c49638f0c67d3,2021/01/01,台北京站門市,2021
4,c0084970c95e1,2021/01/01,台北京站門市,2021


##  Step 3: Create SQLite In-Memory Database

We use SQLite for fast local querying with SQL syntax.


In [ ]:
import sqlite3

# Create an in-memory SQLite DB
conn = sqlite3.connect(':memory:')


##  Step 4: Load DataFrames into SQL Tables

We push the pandas DataFrames into SQLite tables to simulate a production SQL environment.


In [ ]:
# Write each DataFrame to a SQL table
customers.to_sql('Customers', conn, index=False)
orders.to_sql('Orders', conn, index=False)
order_details.to_sql('OrderDetails', conn, index=False)
products.to_sql('Products', conn, index=False)
channels.to_sql('Channels', conn, index=False)


13

##  Step 5.1: Retention Rate by Channel and Year

This query calculates the core metric: the percentage of **new customers** (acquired in 2021 or 2022) who made a **second purchase within the same year**.  
The goal is to benchmark return behavior across acquisition channels.

It answers:  
> “Among customers acquired in a given year and channel, how many made a repeat purchase within that same year?”


In [ ]:
query_1 = """
WITH new_customers AS (
  SELECT
    CustomerId,
    FirstTransactionDate,
    FirstTransactionYear,
    FirstChannel
  FROM Customers
  WHERE FirstTransactionYear IN (2021, 2022)
),
return_stats AS (
  SELECT
    nc.FirstTransactionYear AS Year,
    nc.FirstChannel AS Channel,
    COUNT(DISTINCT nc.CustomerId) AS TotalNewCustomers,
    COUNT(DISTINCT o.CustomerId) AS ReturningCustomers,
    ROUND(COUNT(DISTINCT o.CustomerId) * 1.0 / COUNT(DISTINCT nc.CustomerId), 2) AS ReturnRate
  FROM new_customers nc
  LEFT JOIN Orders o
    ON nc.CustomerId = o.CustomerId
    AND o.TransactionYear = nc.FirstTransactionYear
    AND o.TransactionDate > nc.FirstTransactionDate
  GROUP BY nc.FirstTransactionYear, nc.FirstChannel
)
SELECT *
FROM return_stats
ORDER BY Year, ReturnRate DESC;
"""

pd.read_sql_query(query_1, conn)


,Year,Channel,TotalNewCustomers,ReturningCustomers,ReturnRate
0,2021,台北統一門市,223,55,0.25
1,2021,高雄巨蛋門市,168,40,0.24
2,2021,台中勤美門市,274,62,0.23
3,2021,新竹巨城門市,299,66,0.22
4,2021,台北京站門市,371,79,0.21
5,2021,台北信義門市,376,70,0.19
6,2021,台北南西門市,277,52,0.19
7,2021,官網,6244,849,0.14
8,2021,台北敦南門市,52,3,0.06
9,2022,台北統一門市,199,69,0.35


###Business Insights
1. Retail stores consistently outperform the website in customer retention, with top channels like 台北統一門市 reaching 35% return in 2022 vs 官網's stagnant 14%.
→ Suggests physical interactions drive stronger loyalty.

2. Return rate improved YoY in several retail locations, notably 台北敦南門市 (6% → 23%) and 台北信義門市 (19% → 31%).
→ Indicates effective local customer experience or CRM engagement.

3. 官網 brings the highest volume of new customers but lowest loyalty, signaling a growth–quality gap.
→ Calls for website UX/CRM optimization or better customer qualification.

##  Step 5.2: Year-over-Year Retention Trends by Channel

This query pivots the result to show **how each channel performed over time** in terms of return rate.  
It reveals whether return behavior is **improving or declining**, and highlights channels that may require deeper attention.

It answers:  
> “Did each channel retain more or fewer customers this year than last year?”


In [ ]:
query_2 = """
WITH return_rates AS (
  SELECT
    nc.FirstTransactionYear AS Year,
    nc.FirstChannel AS Channel,
    COUNT(DISTINCT nc.CustomerId) AS TotalNewCustomers,
    COUNT(DISTINCT o.CustomerId) AS ReturningCustomers,
    ROUND(COUNT(DISTINCT o.CustomerId) * 1.0 / COUNT(DISTINCT nc.CustomerId), 2) AS ReturnRate
  FROM Customers nc
  LEFT JOIN Orders o
    ON nc.CustomerId = o.CustomerId
    AND o.TransactionYear = nc.FirstTransactionYear
    AND o.TransactionDate > nc.FirstTransactionDate
  WHERE nc.FirstTransactionYear IN (2021, 2022)
  GROUP BY nc.FirstTransactionYear, nc.FirstChannel
),
pivoted AS (
  SELECT
    Channel,
    MAX(CASE WHEN Year = 2021 THEN ReturnRate END) AS ReturnRate_2021,
    MAX(CASE WHEN Year = 2022 THEN ReturnRate END) AS ReturnRate_2022
  FROM return_rates
  GROUP BY Channel
)
SELECT *,
  ROUND(ReturnRate_2022 - ReturnRate_2021, 2) AS YoY_Change
FROM pivoted
ORDER BY YoY_Change;
"""

pd.read_sql_query(query_2, conn)


,Channel,ReturnRate_2021,ReturnRate_2022,YoY_Change
0,台中中港門市,NaN,0.25,NaN
1,官網,0.14,0.14,0.00
2,台北京站門市,0.21,0.23,0.02
3,新竹巨城門市,0.22,0.24,0.02
4,高雄巨蛋門市,0.24,0.29,0.05
5,台中勤美門市,0.23,0.31,0.08
6,台北南西門市,0.19,0.27,0.08
7,台北統一門市,0.25,0.35,0.10
8,台北信義門市,0.19,0.31,0.12
9,台北敦南門市,0.06,0.23,0.17


###Business Insights
1. 台北敦南門市 showed the largest YoY improvement in return rate (+17%), indicating highly effective local retention tactics or a turnaround in customer experience.

2. Most retail channels improved by 5–12%, especially 台北信義、統一、南西門市, suggesting offline CRM strategies or service delivery are working well.

3. 官網 return rate remained flat at 14%, despite growth in customer volume, reinforcing the need to address loyalty gaps in the digital customer journey.1.

##  Step 5.3: Performance Tiers and Flags

This query adds a **performance tier** to each channel/year pair.  
It categorizes each based on return rate, providing easy-to-digest insight for **CRM targeting or resource allocation**.

It answers:  
> “Which segments are underperforming and should be prioritized for improvement?”


In [ ]:
query_3 = """
WITH new_customers AS (
  SELECT
    CustomerId,
    FirstTransactionDate,
    FirstTransactionYear,
    FirstChannel
  FROM Customers
  WHERE FirstTransactionYear IN (2021, 2022)
),
return_stats AS (
  SELECT
    nc.FirstTransactionYear AS Year,
    nc.FirstChannel AS Channel,
    COUNT(DISTINCT nc.CustomerId) AS TotalNewCustomers,
    COUNT(DISTINCT o.CustomerId) AS ReturningCustomers,
    ROUND(COUNT(DISTINCT o.CustomerId) * 1.0 / COUNT(DISTINCT nc.CustomerId), 2) AS ReturnRate
  FROM new_customers nc
  LEFT JOIN Orders o
    ON nc.CustomerId = o.CustomerId
    AND o.TransactionYear = nc.FirstTransactionYear
    AND o.TransactionDate > nc.FirstTransactionDate
  GROUP BY nc.FirstTransactionYear, nc.FirstChannel
)
SELECT *,
  CASE
    WHEN ReturnRate < 0.3 THEN '⚠️ Underperforming'
    WHEN ReturnRate < 0.4 THEN '🟡 Moderate'
    ELSE '✅ Strong'
  END AS PerformanceTier
FROM return_stats
ORDER BY Year, ReturnRate;
"""

pd.read_sql_query(query_3, conn)


,Year,Channel,TotalNewCustomers,ReturningCustomers,ReturnRate,PerformanceTier
0,2021,台北敦南門市,52,3,0.06,⚠️ Underperforming
1,2021,官網,6244,849,0.14,⚠️ Underperforming
2,2021,台北信義門市,376,70,0.19,⚠️ Underperforming
3,2021,台北南西門市,277,52,0.19,⚠️ Underperforming
4,2021,台北京站門市,371,79,0.21,⚠️ Underperforming
5,2021,新竹巨城門市,299,66,0.22,⚠️ Underperforming
6,2021,台中勤美門市,274,62,0.23,⚠️ Underperforming
7,2021,高雄巨蛋門市,168,40,0.24,⚠️ Underperforming
8,2021,台北統一門市,223,55,0.25,⚠️ Underperforming
9,2022,官網,7306,988,0.14,⚠️ Underperforming


### Business Insights
1. All 2021 channels were underperforming, with return rates below 30%, highlighting a systemic need to strengthen early retention strategies during that year.

2. Website (官網) consistently underperformed across both years despite high acquisition volume, signaling a weak onboarding or post-purchase engagement process.

3. By 2022, some retail channels improved into the “Moderate” tier (≥0.31) — notably 台北信義、統一、勤美 — suggesting successful local CRM or service interventions.

##  Step 6: Business Insights

- **Website return rate declined** from 28% (2021) to 25% (2022), despite more new customers.
- **Retail stores had stronger customer retention**, though slightly decreased in 2022.
- Return behavior highlights the importance of **first purchase channel** in predicting long-term loyalty.

---

##  Suggested Actions

1. Improve the **online experience**: simplify UI/UX, reduce checkout friction.
2. Launch **targeted re-engagement campaigns** for first-time online customers.
3. Explore **behavioral differences by product or acquisition source**.

---

##  Notebook Complete

This concludes our SQL-based analysis for Greenvines’ new customer return behavior.  
We successfully turned business questions into measurable SQL metrics and identified actionable insights for growth.

